In [ ]:
import sqlite3

# Create a SQL connection to our SQLite database
DB_Connection = sqlite3.connect("Users/mahesh/Data Engineer - Assignment Database.db")

cur = DB_Connection.cursor()


In [ ]:
import pandas as pd
df_Sales = pd.read_sql_query("SELECT * from Sales", DB_Connection)
df_customer = pd.read_sql_query("SELECT * from Customers", DB_Connection)
df_Orders = pd.read_sql_query("SELECT * from Orders", DB_Connection)
df_Items = pd.read_sql_query("SELECT * from Items", DB_Connection)

In [ ]:
print("Numbers of rows in Customer df :")
print(df_customer.count())
print("-"*66)
print("Numbers of rows in Orders df :")
print(df_Orders.count())
print("-"*66)
print("Numbers of rows in Sales df :")
print(df_Sales.count())
print("-"*66)
print("Numbers of rows in Items df :")
print(df_Items.count())

In [ ]:
cust_18_35_df = df_customer[(df_customer["age"].between(18, 35))].reset_index()
cust_18_35_df.drop('index',axis=1).head()

In [ ]:
cust_18_35_df.count()

In [ ]:
cust_sales_df = pd.merge(cust_18_35_df, df_Sales, on='customer_id')
cust_sales_df=cust_sales_df.drop('index',axis = 1)
cust_sales_df

In [ ]:
cust_sales_df.count()

In [ ]:
final_df = pd.merge(cust_sales_df, df_Orders, on='sales_id', how='inner')
final_df = final_df[final_df['quantity'].notnull()]
final_df= final_df.reset_index().drop('index',axis=1)
final_df.head(25)

In [ ]:
Customer_ItemQty = final_df.groupby(['customer_id', 'age','item_id'])['quantity'].sum().reset_index()

Customer_ItemQty = pd.merge(Customer_ItemQty,df_Items, on ='item_id').drop('item_id',axis = 1)
Customer_ItemQty


In [ ]:
df_sorted = Customer_ItemQty.sort_values(by='customer_id')
Result = df_sorted[['customer_id','age','item_name','quantity']].reset_index()
Result = Result.drop('index',axis=1)
Result['quantity'] = Result['quantity'].astype(int)
Result

In [ ]:
file_path = 'Users/mahesh/Output.csv'
Result.to_csv(file_path, sep=';',index=False)

# Answer with the SQL Approach

In [ ]:
import sqlite3

# Create a SQL connection to our SQLite database
DB_Connection = sqlite3.connect("Users/bharath/Downloads/Data Engineer - Assignment Database.db")

cur = DB_Connection.cursor()

print('customer_id,','age,','Item_id,','Total_quantity')
Query = '''
SELECT A.customer_id, A.age, 
       CASE 
           WHEN B.item_id = 1 THEN 'X'
           WHEN B.item_id = 2 THEN 'Y'
           WHEN B.item_id = 3 THEN 'Z'
       END AS item_name,
       SUM(B.quantity) AS total_quantity
FROM (
    SELECT c.customer_id, c.age, s.sales_id 
    FROM customers c
    JOIN sales s ON c.customer_id = s.customer_id 
    WHERE c.age >= 18 AND c.age <= 35
) A
JOIN Orders B ON A.sales_id = B.sales_id
WHERE B.quantity IS NOT NULL
GROUP BY A.customer_id, A.age, item_name
ORDER BY A.customer_id;'''

for row in cur.execute(Query):
    print(row)
